# Setup

In [ ]:
# General
library(tidyverse) # incl. ggplot2
library(reticulate)
use_python("/home/michi/Software/venvs/scAnalysis_1.8_R4.1/bin/python3", required = TRUE)

# Analysis
library(Signac)
library(Seurat)
library(SeuratDisk)
library(SingleCellExperiment)
library(pastecs)

# Plotting
library(cowplot)

#Set theme for plotting
##########################################
#theme_set(theme_bw())
theme_set(theme_cowplot())

# Borders around cowplots:
currentTheme <- theme_get()
currentTheme <- theme_replace(aspect.ratio = 1,
                              panel.border = element_rect(colour = "black", size = 0.75, fill = NA),
                              rect = element_rect(colour = "black", size = 0.5, linetype = 1, fill = "transparent"),
                              panel.background = element_rect(colour = "black", size = 0.5))
currentTheme <- theme_update(plot.title = element_text(hjust = 0.5))

library(future)
plan("multicore", workers=64)
options(future.globals.maxSize = 100 * 1024 ^ 3) # for 50 Gb RAM

# source("~/Scripts/R-Functions.R")
# 
# # Parallelization
# library(BiocParallel)
# register(MulticoreParam(32, progressbar = TRUE))

set.seed(1234)

In [ ]:
%%python
# General
import scipy as sci
import numpy as np
import pandas as pd
import logging
import scipy.stats

# Analysis
import scanpy as sc
sc.logging.print_versions()

# Doublets Removed
## Load Data

In [ ]:
# read preprocessed data
seurat <- readRDS("/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_seurat_jointPeaks.rds")

In [ ]:
%%python
filtered = sc.read('/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets.h5ad')

filtered_var=list(filtered.var_names)

In [ ]:
# Filter genes
seurat[["RNA"]] <- subset(seurat[["RNA"]], features = py$filtered_var)
seurat[["RNA"]] 

In [ ]:
seurat@meta.data$sample <- sapply(rownames(seurat@meta.data), function(x){strsplit(x,"_")[[1]][1]})
Idents(seurat) <- seurat@meta.data$sample

## Preprocessing

In [ ]:
DefaultAssay(seurat) <- "RNA"
seurat <- SCTransform(seurat, verbose = TRUE, return.only.var.genes = FALSE, variable.features.n = NULL, vst.flavor = "v2")
seurat <- RunPCA(seurat)
seurat <- RunUMAP(seurat, dims = 1:50, reduction.name = 'umap.rna', reduction.key = 'rnaUMAP_')

In [ ]:
DefaultAssay(seurat) <- "peaks"
seurat <- RunTFIDF(seurat)
seurat <- FindTopFeatures(seurat, min.cutoff = 'q0')
seurat <- RunSVD(seurat)
seurat <- RunUMAP(seurat, reduction = 'lsi', dims = 2:50, reduction.name = "umap.atac", reduction.key = "atacUMAP_")

## Weighted Nearest Neighbor Analysis

In [ ]:
# build a joint neighbor graph using both assays
seurat <- FindMultiModalNeighbors(
  object = seurat,
  reduction.list = list("pca", "lsi"), 
  dims.list = list(1:50, 2:50),
  modality.weight.name = "RNA.weight",
  verbose = TRUE
)

# build a joint UMAP visualization
DefaultAssay(seurat) <- "SCT"
seurat <- RunUMAP(
  object = seurat,
  nn.name = "weighted.nn",
  assay = "SCT",
  verbose = TRUE
)

seurat <- RunSPCA(seurat, assay = 'SCT', graph = 'wsnn', npcs = 50)

In [ ]:
DimPlot(seurat, label = TRUE, repel = TRUE, reduction = "umap.rna") + NoLegend() + ggtitle("RNA")
DimPlot(seurat, label = TRUE, repel = TRUE, reduction = "umap.atac") + NoLegend() + ggtitle("ATAC")
DimPlot(seurat, label = TRUE, repel = TRUE, reduction = "umap") + NoLegend() + ggtitle("WNN")

## Export Results

In [ ]:
saveRDS(seurat, "/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_seurat_jointPeaks_sctNormalized_wnn.rds")

In [ ]:
seurat_export <- seurat
Assays(seurat_export)
DefaultAssay(seurat_export) <- 'SCT'
#seurat_export[["RNA"]] <- NULL
seurat_export[["ATAC"]] <- NULL
seurat_export[["peaks"]] <- NULL

SaveH5Seurat(seurat_export, filename = "/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_wsnn.h5Seurat", overwrite = TRUE)
Convert("/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_wsnn.h5Seurat", dest = "h5ad", overwrite = TRUE)

seurat_export@graphs$wsnn <- NULL

SaveH5Seurat(seurat_export, filename = "/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_wknn.h5Seurat", overwrite = TRUE)
Convert("/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_wknn.h5Seurat", dest = "h5ad", overwrite = TRUE)

In [ ]:
# Add feature meta data (since Seurat v4 -> will be fixed?)
var <- c('detection_rate','gmean', 'variance', 'residual_variance')
seurat_export[["SCT"]]@meta.features <- SCTResults(seurat_export[["SCT"]], slot = "feature.attributes")[, var]
seurat_export[["SCT"]]@meta.features$variable <- FALSE
seurat_export[["SCT"]]@meta.features[VariableFeatures(seurat_export[["SCT"]] ), "variable"] <- TRUE
colnames(seurat_export[["SCT"]]@meta.features) <- paste0("sct.", colnames(seurat_export[["SCT"]]@meta.features) )

# Convert to SingleCellExperiment
sce_export <- as.SingleCellExperiment(seurat_export)

# Add feature meta data (since Seurat v4 -> will be fixed?)
rowData(sce_export) <- seurat_export[["SCT"]]@meta.features

# Rename and add layers
SummarizedExperiment::assay(sce_export, i = 1) <- seurat_export[["SCT"]]@counts
SummarizedExperiment::assay(sce_export, i = 2) <- seurat_export[["SCT"]]@data
SummarizedExperiment::assay(sce_export, i = 3) <- seurat_export[["SCT"]]@scale.data
#SummarizedExperiment::assay(sce_export, i = 4) <- seurat_export[["RNA"]]@counts[rownames(seurat_export[["RNA"]]@counts),]
SummarizedExperiment::assayNames(sce_export) <- c("sct_counts", "sct_logcounts", "sct_scale_data") #, "raw_counts")

# Save to file
saveRDS(seurat_export, "/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_seurat.rds")
saveRDS(sce_export, "/home/michi/Projects/scMultiome_NVF_Mouse-Pancreas-Development_E14.5_Notebooks/Files/E14-E15_adata_filtered_rmDoublets_sctNormalized_sce.rds")

# Session Info

In [ ]:
sessionInfo()